# <span style='color:#ff5f27'> Initialization </span>

### Hopsworks Settings

In [1]:
import sys
from pathlib import Path
import os

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
if os.path.exists(f"{root_dir}/.env"):
    settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /home/federica_lorenzini/sml-bike-sharing
HopsworksSettings initialized!


### Imports

In [2]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs import util
from mlfs import config
import json
import warnings
warnings.filterwarnings("ignore")

2026-01-11 15:30:47,496 WARNING: DeprecationWarning: 'parseString' deprecated - use 'parse_string'

2026-01-11 15:30:47,524 WARNING: DeprecationWarning: 'resetCache' deprecated - use 'reset_cache'

2026-01-11 15:30:47,717 WARNING: DeprecationWarning: 'enablePackrat' deprecated - use 'enable_packrat'

2026-01-11 15:30:47,800 WARNING: In /home/federica_lorenzini/sml-bike-sharing/.venv/lib/python3.10/site-packages/matplotlib/mpl-data/stylelib/classic.mplstyle: 'parseString' deprecated - use 'parse_string'
2026-01-11 15:30:47,801 WARNING: In /home/federica_lorenzini/sml-bike-sharing/.venv/lib/python3.10/site-packages/matplotlib/mpl-data/stylelib/classic.mplstyle: 'resetCache' deprecated - use 'reset_cache'


# <span style='color:#ff5f27'> Retrieve Historical Data </span>

### Retrieve Metadata

In [3]:
project = hopsworks.login()
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

location_str = secrets.get_secret("BIKES_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
latitude=location['latitude']
longitude=location['longitude']
today = datetime.date.today()
yesterday = today - datetime.timedelta(days=1)

print(f"City: {city}")
print(f"Contry: {country}")
print(f"Latitude: {latitude}")
print(f"Longitude: {longitude}")
print(f"Day: {today}")

2026-01-11 15:30:49,757 INFO: Initializing external client
2026-01-11 15:30:49,760 INFO: Base URL: https://c.app.hopsworks.ai:443


2026-01-11 15:30:51,986 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279175
City: Trento
Contry: Italy
Latitude: 46.07
Longitude: 11.12
Day: 2026-01-11


### Connect to Feature Groups

In [4]:
bikes_fg = fs.get_feature_group(
    name='bikes_trento',
    version=2,
)
weather_fg = fs.get_feature_group(
    name='weather_trento',
    version=2,
)

# <span style='color:#ff5f27'> Get Today's Data </span>

### Bikes Data

In [5]:
# No API-key is required for this API

network_id = "e-motion-trento"
url = f"https://api.citybik.es/v2/networks/{network_id}"

try:
    resp = requests.get(url).json()
    print("Request Successful!")
except:
    print("Something went wrong, please check the URL.")
    print("Networn Answer:")
    print(requests.get(url))


Request Successful!


Check The following info to be the same as your request

In [6]:
print(f"Network ID: {resp['network']['id']}")

print(f"City: {resp['network']['location']['city']}")
print(f"Contry: {resp['network']['location']['country']}")
print(f"Latitude: {resp['network']['location']['latitude']}")
print(f"Longitude: {resp['network']['location']['longitude']}")

Network ID: e-motion-trento
City: Trento
Contry: IT
Latitude: 46.06643205823519
Longitude: 11.122145390351879


Data Preview

In [7]:
stations_df = pd.DataFrame(resp['network']['stations'])

stations_df['city'] = city
stations_df['country'] = country
stations_df['tag'] = 'e-motion-trento'

stations_df['timestamp'] = pd.to_datetime(stations_df['timestamp'], format='%Y-%m-%dT%H:%M:%S.%f%zZ')
stations_df = stations_df.rename(columns={'timestamp': 'date', 'free_bikes': 'bikes', 'empty_slots': 'free'})

# Convert int64 to int32 as Hopsworks don't parse
stations_df['bikes'] = stations_df['bikes'].astype('int32')
stations_df['free'] = stations_df['free'].astype('int32')
# Extra needs to be string
stations_df['extra'] = stations_df['extra'].astype('str')

stations_df.head()

,id,name,latitude,longitude,date,bikes,free,extra,city,country,tag
0,00d841f5a7a286ba0717de688b6994a2,10.02 Top Center,46.090104,11.118584,2026-01-11 14:30:36.320882+00:00,1,13,"{'uid': '1122', 'number': 1002, 'status': 'onl...",Trento,Italy,e-motion-trento
1,01d99ffbc2e95da5b97cd33ceacb6e95,20.10 Noriglio,45.883685,11.070397,2026-01-11 14:30:36.321307+00:00,4,4,"{'uid': '1774', 'number': 2010, 'status': 'onl...",Trento,Italy,e-motion-trento
2,06e5996f6846e61e53875c27afd20722,20.09 Sacco,45.884737,11.023139,2026-01-11 14:30:36.321302+00:00,5,6,"{'uid': '1773', 'number': 2009, 'status': 'onl...",Trento,Italy,e-motion-trento
3,0b05bb893eb6aa4ebc6d49b0f07feb0b,10.18 Vannetti,46.072997,11.121911,2026-01-11 14:30:36.321015+00:00,0,0,"{'uid': '1494', 'number': 1018, 'status': 'off...",Trento,Italy,e-motion-trento
4,0d975b31512ff242354da4c87d9a603b,11.01 Ospedale San Giovanni,46.212476,11.094596,2026-01-11 14:30:36.321132+00:00,3,9,"{'uid': '1740', 'number': 1101, 'status': 'onl...",Trento,Italy,e-motion-trento


In [8]:
stations_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype              
---  ------     --------------  -----              
 0   id         85 non-null     object             
 1   name       85 non-null     object             
 2   latitude   85 non-null     float64            
 3   longitude  85 non-null     float64            
 4   date       85 non-null     datetime64[ns, UTC]
 5   bikes      85 non-null     int32              
 6   free       85 non-null     int32              
 7   extra      85 non-null     object             
 8   city       85 non-null     object             
 9   country    85 non-null     object             
 10  tag        85 non-null     object             
dtypes: datetime64[ns, UTC](1), float64(2), int32(2), object(6)
memory usage: 6.8+ KB


### Weather Data

In [9]:
weather_daily_df = util.get_daily_weather_forecast(city=city, latitude=latitude, longitude=longitude)
weather_daily_df

Coordinates 46.0°N 11.0°E
Elevation 193.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,weather_code,apparent_temperature_mean,daylight_duration,precipitation_sum,wind_speed_10m_max,city
0,2026-01-11,2,-2.276061,32165.906250,0.000000,5.991594,Trento
1,2026-01-12,3,-3.405265,32267.652344,0.000000,3.319036,Trento
2,2026-01-13,3,0.622029,32373.125000,0.000000,2.276840,Trento
3,2026-01-14,51,3.051547,32482.150391,0.400000,2.902413,Trento
4,2026-01-15,51,4.228703,32594.542969,0.400000,2.545584,Trento
5,2026-01-16,51,3.586735,32710.150391,0.300000,3.600000,Trento
6,2026-01-17,53,4.472055,32828.796875,6.799999,3.396233,Trento


In [10]:
weather_daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   date                       7 non-null      datetime64[ns]
 1   weather_code               7 non-null      int32         
 2   apparent_temperature_mean  7 non-null      float32       
 3   daylight_duration          7 non-null      float32       
 4   precipitation_sum          7 non-null      float32       
 5   wind_speed_10m_max         7 non-null      float32       
 6   city                       7 non-null      object        
dtypes: datetime64[ns](1), float32(4), int32(1), object(1)
memory usage: 380.0+ bytes


# <span style='color:#ff5f27'> Upload Data </span>

In [11]:
bikes_fg.insert(stations_df)
print("Updated bike's feature group.")
weather_fg.insert(weather_daily_df, wait=True)
print("Updated weather's feature group.")

2026-01-11 15:30:55,592 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279175/fs/1265791/fg/1893939


Uploading Dataframe: 100.00% |██████████| Rows 85/85 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: bikes_trento_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279175/jobs/named/bikes_trento_2_offline_fg_materialization/executions
Updated bike's feature group.
2026-01-11 15:31:10,966 INFO: 	5 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279175/fs/1265791/fg/1893940


Uploading Dataframe: 100.00% |██████████| Rows 7/7 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: weather_trento_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279175/jobs/named/weather_trento_2_offline_fg_materialization/executions
2026-01-11 15:31:28,576 INFO: Waiting for execution to finish. Current state: INITIALIZING. Final status: UNDEFINED
2026-01-11 15:31:31,769 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2026-01-11 15:31:34,965 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2026-01-11 15:33:14,240 INFO: Waiting for log aggregation to finish.
2026-01-11 15:33:40,913 INFO: Execution finished successfully.
Updated weather's feature group.
